In [64]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

import random

In [65]:
train_path = '../data/interim/train_final_ver1.csv'
test_path = '../data/interim/test_final_ver1.csv'

df_train = pd.read_csv(train_path, index_col=0)
df_test = pd.read_csv(test_path, index_col=0)

In [66]:
subway_path = '../data/external/subway_feature.csv'

df_subway = pd.read_csv(subway_path)

display(df_subway)

,역사_ID,역사명,호선,위도,경도
0,9996,미사,5호선,37.560927,127.193877
1,9995,강일,5호선,37.557490,127.175930
2,4929,김포공항,김포골드라인,37.562360,126.801868
3,4928,고촌,김포골드라인,37.601243,126.770345
4,4927,풍무,김포골드라인,37.612488,126.732387
...,...,...,...,...,...
763,154,종로5가,1호선,37.570926,127.001849
764,153,종로3가,1호선,37.570406,126.991847
765,152,종각,1호선,37.570161,126.982923
766,151,시청,1호선,37.565715,126.977088


In [67]:
df_subway = df_subway.rename(columns={'위도' : 'sub_y', '경도' : 'sub_x'})
display(df_subway)

,역사_ID,역사명,호선,sub_y,sub_x
0,9996,미사,5호선,37.560927,127.193877
1,9995,강일,5호선,37.557490,127.175930
2,4929,김포공항,김포골드라인,37.562360,126.801868
3,4928,고촌,김포골드라인,37.601243,126.770345
4,4927,풍무,김포골드라인,37.612488,126.732387
...,...,...,...,...,...
763,154,종로5가,1호선,37.570926,127.001849
764,153,종로3가,1호선,37.570406,126.991847
765,152,종각,1호선,37.570161,126.982923
766,151,시청,1호선,37.565715,126.977088


In [68]:
df_train['is_test'] = 0
df_test['is_test'] = 1
df_all = pd.concat([df_train, df_test])
display(df_all)

,복도유형,난방방식,시군구,전용면적,계약일,층,건축년도,k-전체동수,k-전체세대수,k-연면적,건축면적,x,y,target,아파트명,계약년월,도로명,is_test
0,계단식,개별난방,서울특별시 강남구 개포동,79.97,8,3,1987,8.0,270.0,22637.0,4858.0,127.05721,37.476763,124000.0,개포6차우성,201712,언주로 3,0
1,계단식,개별난방,서울특별시 강남구 개포동,79.97,22,4,1987,8.0,270.0,22637.0,4858.0,127.05721,37.476763,123500.0,개포6차우성,201712,언주로 3,0
2,계단식,개별난방,서울특별시 강남구 개포동,54.98,28,5,1987,8.0,270.0,22637.0,4858.0,127.05721,37.476763,91500.0,개포6차우성,201712,언주로 3,0
3,계단식,개별난방,서울특별시 강남구 개포동,79.97,3,4,1987,8.0,270.0,22637.0,4858.0,127.05721,37.476763,130000.0,개포6차우성,201801,언주로 3,0
4,계단식,개별난방,서울특별시 강남구 개포동,79.97,8,2,1987,8.0,270.0,22637.0,4858.0,127.05721,37.476763,117000.0,개포6차우성,201801,언주로 3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9267,혼합식,개별난방,서울특별시 중랑구 신내동,84.65,19,13,2014,16.0,1402.0,190866.0,14171.0,127.10672,37.618870,NaN,신내우디안1단지,202307,신내역로1길 85,1
9268,혼합식,개별난방,서울특별시 중랑구 신내동,84.62,25,12,2014,16.0,1402.0,190866.0,14171.0,127.10672,37.618870,NaN,신내우디안1단지,202307,신내역로1길 85,1
9269,혼합식,개별난방,서울특별시 중랑구 신내동,101.65,27,12,2014,16.0,1402.0,190866.0,14171.0,127.10672,37.618870,NaN,신내우디안1단지,202308,신내역로1길 85,1
9270,혼합식,개별난방,서울특별시 중랑구 신내동,84.94,2,18,2014,16.0,1402.0,190866.0,14171.0,127.10672,37.618870,NaN,신내우디안1단지,202309,신내역로1길 85,1


In [69]:
display(df_all.describe())
display(df_subway.describe())

,전용면적,계약일,층,건축년도,k-전체동수,k-전체세대수,k-연면적,건축면적,x,y,target,계약년월,is_test
count,1.128094e+06,1.128094e+06,1.128094e+06,1.128094e+06,250887.000000,251969.000000,2.519690e+05,2.518150e+05,1.128094e+06,1.128094e+06,1.118822e+06,1.128094e+06,1.128094e+06
mean,7.716028e+01,1.579966e+01,8.881412e+00,1.998791e+03,14.810871,1186.767436,1.619010e+05,1.904915e+05,1.269943e+02,3.755437e+01,5.799153e+04,2.014829e+05,8.219173e-03
std,2.936448e+01,8.722162e+00,5.989302e+00,9.358540e+00,17.682771,1197.452046,1.843113e+05,1.735029e+06,7.329411e-01,2.224538e-01,4.642602e+04,4.237691e+02,9.028635e-02
min,1.002000e+01,1.000000e+00,-4.000000e+00,1.961000e+03,1.000000,59.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.500000e+02,2.007010e+05,0.000000e+00
25%,5.965000e+01,8.000000e+00,4.000000e+00,1.992000e+03,5.000000,405.000000,5.366600e+04,0.000000e+00,1.269175e+02,3.750836e+01,3.050000e+04,2.011110e+05,0.000000e+00
50%,8.187000e+01,1.600000e+01,8.000000e+00,2.000000e+03,10.000000,768.000000,1.026240e+05,1.735000e+03,1.270199e+02,3.754750e+01,4.480000e+04,2.015080e+05,0.000000e+00
75%,8.496000e+01,2.300000e+01,1.200000e+01,2.005000e+03,17.000000,1622.000000,2.039040e+05,8.414210e+03,1.270675e+02,3.760063e+01,6.980000e+04,2.018050e+05,0.000000e+00
max,4.243200e+02,3.100000e+01,6.900000e+01,2.023000e+03,124.000000,9510.000000,9.591851e+06,3.159620e+07,1.271800e+02,3.768929e+01,1.450000e+06,2.023090e+05,1.000000e+00


,역사_ID,sub_y,sub_x
count,768.000000,768.000000,768.000000
mean,2297.946615,37.509654,126.969929
std,1418.164103,0.151260,0.183336
min,150.000000,36.769502,126.441442
25%,1276.750000,37.470875,126.851355
50%,1955.500000,37.521240,126.993792
75%,3131.250000,37.575834,127.070481
max,9996.000000,37.948100,127.723792


In [70]:
df_all.loc[(df_all['아파트명'] == '삼성') & (df_all['x'] < 1), 'x'] = 126.9788
df_all.loc[(df_all['아파트명'] == '삼성') & (df_all['y'] < 1), 'y'] = 37.6133

display(df_all.describe())
display(df_subway.describe())

,전용면적,계약일,층,건축년도,k-전체동수,k-전체세대수,k-연면적,건축면적,x,y,target,계약년월,is_test
count,1.128094e+06,1.128094e+06,1.128094e+06,1.128094e+06,250887.000000,251969.000000,2.519690e+05,2.518150e+05,1.128094e+06,1.128094e+06,1.118822e+06,1.128094e+06,1.128094e+06
mean,7.716028e+01,1.579966e+01,8.881412e+00,1.998791e+03,14.810871,1186.767436,1.619010e+05,1.904915e+05,1.269984e+02,3.755560e+01,5.799153e+04,2.014829e+05,8.219173e-03
std,2.936448e+01,8.722162e+00,5.989302e+00,9.358540e+00,17.682771,1197.452046,1.843113e+05,1.735029e+06,9.067939e-02,5.680886e-02,4.642602e+04,4.237691e+02,9.028635e-02
min,1.002000e+01,1.000000e+00,-4.000000e+00,1.961000e+03,1.000000,59.000000,0.000000e+00,0.000000e+00,1.267983e+02,3.743440e+01,3.500000e+02,2.007010e+05,0.000000e+00
25%,5.965000e+01,8.000000e+00,4.000000e+00,1.992000e+03,5.000000,405.000000,5.366600e+04,0.000000e+00,1.269175e+02,3.750840e+01,3.050000e+04,2.011110e+05,0.000000e+00
50%,8.187000e+01,1.600000e+01,8.000000e+00,2.000000e+03,10.000000,768.000000,1.026240e+05,1.735000e+03,1.270199e+02,3.754750e+01,4.480000e+04,2.015080e+05,0.000000e+00
75%,8.496000e+01,2.300000e+01,1.200000e+01,2.005000e+03,17.000000,1622.000000,2.039040e+05,8.414210e+03,1.270675e+02,3.760077e+01,6.980000e+04,2.018050e+05,0.000000e+00
max,4.243200e+02,3.100000e+01,6.900000e+01,2.023000e+03,124.000000,9510.000000,9.591851e+06,3.159620e+07,1.271800e+02,3.768929e+01,1.450000e+06,2.023090e+05,1.000000e+00


,역사_ID,sub_y,sub_x
count,768.000000,768.000000,768.000000
mean,2297.946615,37.509654,126.969929
std,1418.164103,0.151260,0.183336
min,150.000000,36.769502,126.441442
25%,1276.750000,37.470875,126.851355
50%,1955.500000,37.521240,126.993792
75%,3131.250000,37.575834,127.070481
max,9996.000000,37.948100,127.723792


In [71]:
import math

# 위경도를 이용해 두 지점간의 거리를 구하는 함수를 생성합니다.
def haversine_distance(lat1, lon1, lat2, lon2): # y1, x1, y2, x2
    radius = 6371.0

    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = radius * c
    return distance

In [72]:
df_loc = df_all[['x', 'y']]
unique_loc = set()
for i in range(len(df_loc)):
    unique_loc.add((df_loc['x'].values[i], df_loc['y'].values[i]))

unique_list = list(unique_loc)

unique_x, unique_y = [], []
for i in range(len(unique_list)):
    x, y = unique_list[i]
    unique_x.append(x)
    unique_y.append(y)

In [73]:
ser_x = pd.Series(unique_x)
ser_y = pd.Series(unique_y)
display(ser_x)
display(ser_y)

0       126.951669
1       127.054015
2       127.047542
3       126.904535
4       127.040462
           ...    
5882    127.033902
5883    127.057734
5884    126.918684
5885    126.894966
5886    127.044834
Length: 5887, dtype: float64

0       37.480261
1       37.489476
2       37.583577
3       37.589763
4       37.590592
          ...    
5882    37.486772
5883    37.539845
5884    37.526537
5885    37.513878
5886    37.625137
Length: 5887, dtype: float64

In [74]:
df_loc = pd.DataFrame({'x' : ser_x, 'y' : ser_y})

display(df_loc)
display(df_subway)

,x,y
0,126.951669,37.480261
1,127.054015,37.489476
2,127.047542,37.583577
3,126.904535,37.589763
4,127.040462,37.590592
...,...,...
5882,127.033902,37.486772
5883,127.057734,37.539845
5884,126.918684,37.526537
5885,126.894966,37.513878


,역사_ID,역사명,호선,sub_y,sub_x
0,9996,미사,5호선,37.560927,127.193877
1,9995,강일,5호선,37.557490,127.175930
2,4929,김포공항,김포골드라인,37.562360,126.801868
3,4928,고촌,김포골드라인,37.601243,126.770345
4,4927,풍무,김포골드라인,37.612488,126.732387
...,...,...,...,...,...
763,154,종로5가,1호선,37.570926,127.001849
764,153,종로3가,1호선,37.570406,126.991847
765,152,종각,1호선,37.570161,126.982923
766,151,시청,1호선,37.565715,126.977088


In [75]:
subway_sample = df_subway.iloc[:10]
loc_sample = df_loc.iloc[:10]

display(subway_sample)
display(loc_sample)

,역사_ID,역사명,호선,sub_y,sub_x
0,9996,미사,5호선,37.560927,127.193877
1,9995,강일,5호선,37.557490,127.175930
2,4929,김포공항,김포골드라인,37.562360,126.801868
3,4928,고촌,김포골드라인,37.601243,126.770345
4,4927,풍무,김포골드라인,37.612488,126.732387
5,4926,사우(김포시청),김포골드라인,37.620249,126.719731
6,4925,걸포북변,김포골드라인,37.631650,126.705975
7,4924,운양,김포골드라인,37.653867,126.683930
8,4923,장기,김포골드라인,37.643986,126.669017
9,4922,마산,김포골드라인,37.640732,126.644344


,x,y
0,126.951669,37.480261
1,127.054015,37.489476
2,127.047542,37.583577
3,126.904535,37.589763
4,127.040462,37.590592
5,126.978091,37.481291
6,126.872099,37.538179
7,126.942149,37.505792
8,127.033288,37.609448
9,127.035487,37.576978


In [76]:
def get_subway_min_dist(df1, df2): # 거리 차이를 계산할 데이터 프레임 --> 전체 데이터, 버스 or 지하철 데이터
    min_dist = []
    for i in range(len(df1)):
        x, y = df1['x'].values[i], df1['y'].values[i]
        dist = []
        for j in range(len(df2)):
            tar_x, tar_y = df2['sub_x'].values[j], df2['sub_y'].values[j]
            harv_dist = haversine_distance(y, x, tar_y, tar_x)
            dist.append(harv_dist)
        min_dist.append(min(dist))

    return min_dist
        

In [77]:
# sample로 결과값이 잘 나오는지  테스트
sub_min_dist = get_subway_min_dist(loc_sample, subway_sample)
print(sub_min_dist)

[16.058474418875416, 13.145090047345583, 11.681155779670387, 9.547070853572544, 12.493212374112847, 17.966432108933997, 6.7500549731014985, 13.876918862330527, 13.833543528870635, 12.566543477505954]


In [78]:
# 전체 데이터셋에 대해 실행
sub_min_dist = get_subway_min_dist(df_loc, df_subway)

In [79]:
df_loc['sub_min_dist'] = sub_min_dist
display(df_loc)

,x,y,sub_min_dist
0,126.951669,37.480261,0.144627
1,127.054015,37.489476,0.204798
2,127.047542,37.583577,0.320424
3,126.904535,37.589763,0.794475
4,127.040462,37.590592,0.367183
...,...,...,...
5882,127.033902,37.486772,0.255192
5883,127.057734,37.539845,0.549306
5884,126.918684,37.526537,0.188387
5885,126.894966,37.513878,0.451294


In [80]:
df_subway = pd.merge(df_all, df_loc, on=['x', 'y'], how='inner')
display(df_subway)

,복도유형,난방방식,시군구,전용면적,계약일,층,건축년도,k-전체동수,k-전체세대수,k-연면적,건축면적,x,y,target,아파트명,계약년월,도로명,is_test,sub_min_dist
0,계단식,개별난방,서울특별시 강남구 개포동,79.97,8,3,1987,8.0,270.0,22637.0,4858.0,127.057210,37.476763,124000.0,개포6차우성,201712,언주로 3,0,1.129775
1,계단식,개별난방,서울특별시 강남구 개포동,79.97,22,4,1987,8.0,270.0,22637.0,4858.0,127.057210,37.476763,123500.0,개포6차우성,201712,언주로 3,0,1.129775
2,계단식,개별난방,서울특별시 강남구 개포동,54.98,28,5,1987,8.0,270.0,22637.0,4858.0,127.057210,37.476763,91500.0,개포6차우성,201712,언주로 3,0,1.129775
3,계단식,개별난방,서울특별시 강남구 개포동,79.97,3,4,1987,8.0,270.0,22637.0,4858.0,127.057210,37.476763,130000.0,개포6차우성,201801,언주로 3,0,1.129775
4,계단식,개별난방,서울특별시 강남구 개포동,79.97,8,2,1987,8.0,270.0,22637.0,4858.0,127.057210,37.476763,117000.0,개포6차우성,201801,언주로 3,0,1.129775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,복도식,개별난방,서울특별시 중랑구 신내동,49.77,22,7,1996,NaN,NaN,NaN,NaN,127.093511,37.616533,NaN,신내6대주,202307,신내로19길 42,1,0.203699
1128090,복도식,개별난방,서울특별시 중랑구 신내동,39.84,7,3,1996,NaN,NaN,NaN,NaN,127.093511,37.616533,NaN,신내6대주,202308,신내로19길 42,1,0.203699
1128091,혼합식,개별난방,서울특별시 중랑구 신내동,59.76,17,4,1996,NaN,NaN,NaN,NaN,127.093511,37.616533,NaN,신내6대주,202308,신내로19길 42,1,0.203699
1128092,복도식,개별난방,서울특별시 중랑구 신내동,39.84,2,9,1996,NaN,NaN,NaN,NaN,127.093511,37.616533,NaN,신내6대주,202309,신내로19길 42,1,0.203699


In [81]:
# 결측치 확인
df_subway.isnull().sum()

복도유형                 0
난방방식                 0
시군구                  0
전용면적                 0
계약일                  0
층                    0
건축년도                 0
k-전체동수          877207
k-전체세대수         876125
k-연면적           876125
건축면적            876279
x                    0
y                    0
target            9272
아파트명              2136
계약년월                 0
도로명                  0
is_test              0
sub_min_dist         0
dtype: int64

In [82]:
subway_train = df_subway[df_subway['is_test'] == 0]
subway_test = df_subway[df_subway['is_test'] == 1]

display(subway_train)
display(subway_test)

,복도유형,난방방식,시군구,전용면적,계약일,층,건축년도,k-전체동수,k-전체세대수,k-연면적,건축면적,x,y,target,아파트명,계약년월,도로명,is_test,sub_min_dist
0,계단식,개별난방,서울특별시 강남구 개포동,79.97,8,3,1987,8.0,270.0,22637.0,4858.0,127.057210,37.476763,124000.0,개포6차우성,201712,언주로 3,0,1.129775
1,계단식,개별난방,서울특별시 강남구 개포동,79.97,22,4,1987,8.0,270.0,22637.0,4858.0,127.057210,37.476763,123500.0,개포6차우성,201712,언주로 3,0,1.129775
2,계단식,개별난방,서울특별시 강남구 개포동,54.98,28,5,1987,8.0,270.0,22637.0,4858.0,127.057210,37.476763,91500.0,개포6차우성,201712,언주로 3,0,1.129775
3,계단식,개별난방,서울특별시 강남구 개포동,79.97,3,4,1987,8.0,270.0,22637.0,4858.0,127.057210,37.476763,130000.0,개포6차우성,201801,언주로 3,0,1.129775
4,계단식,개별난방,서울특별시 강남구 개포동,79.97,8,2,1987,8.0,270.0,22637.0,4858.0,127.057210,37.476763,117000.0,개포6차우성,201801,언주로 3,0,1.129775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1126418,혼합식,개별난방,서울특별시 강동구 둔촌동,73.98,2,2,1995,NaN,NaN,NaN,NaN,127.141232,37.530097,23800.0,성원,200707,양재대로102가길 22,0,0.509011
1126419,혼합식,개별난방,서울특별시 강동구 둔촌동,73.98,25,11,1995,NaN,NaN,NaN,NaN,127.141232,37.530097,26000.0,성원,200709,양재대로102가길 22,0,0.509011
1126420,계단식,지역난방,서울특별시 서초구 반포동,84.07,30,5,1979,NaN,NaN,NaN,NaN,126.996770,37.499668,100000.0,미주,200708,신반포로16길 15-20,0,0.423304
1126421,혼합식,지역난방,서울특별시 송파구 송파동,74.04,7,10,1995,NaN,NaN,NaN,NaN,127.106904,37.509822,31000.0,호수임광,200705,송파대로48길,0,0.472486


,복도유형,난방방식,시군구,전용면적,계약일,층,건축년도,k-전체동수,k-전체세대수,k-연면적,건축면적,x,y,target,아파트명,계약년월,도로명,is_test,sub_min_dist
101,계단식,개별난방,서울특별시 강남구 개포동,79.97,26,5,1987,8.0,270.0,22637.0,4858.00,127.057210,37.476763,NaN,개포6차우성,202307,언주로 3,1,1.129775
261,혼합식,지역난방,서울특별시 강남구 개포동,161.00,28,15,1984,5.0,405.0,63304.0,61064.24,127.055990,37.483894,NaN,개포우성3차,202307,개포로 307,1,0.413746
262,혼합식,지역난방,서울특별시 강남구 개포동,133.46,10,14,1984,5.0,405.0,63304.0,61064.24,127.055990,37.483894,NaN,개포우성3차,202308,개포로 307,1,0.413746
263,혼합식,지역난방,서울특별시 강남구 개포동,104.43,18,6,1984,5.0,405.0,63304.0,61064.24,127.055990,37.483894,NaN,개포우성3차,202308,개포로 307,1,0.413746
3669,복도식,지역난방,서울특별시 강남구 개포동,74.25,28,8,1983,6.0,940.0,93810.0,6524.00,127.068028,37.487802,NaN,개포주공5단지,202307,삼성로4길 17,1,0.221601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128089,복도식,개별난방,서울특별시 중랑구 신내동,49.77,22,7,1996,NaN,NaN,NaN,NaN,127.093511,37.616533,NaN,신내6대주,202307,신내로19길 42,1,0.203699
1128090,복도식,개별난방,서울특별시 중랑구 신내동,39.84,7,3,1996,NaN,NaN,NaN,NaN,127.093511,37.616533,NaN,신내6대주,202308,신내로19길 42,1,0.203699
1128091,혼합식,개별난방,서울특별시 중랑구 신내동,59.76,17,4,1996,NaN,NaN,NaN,NaN,127.093511,37.616533,NaN,신내6대주,202308,신내로19길 42,1,0.203699
1128092,복도식,개별난방,서울특별시 중랑구 신내동,39.84,2,9,1996,NaN,NaN,NaN,NaN,127.093511,37.616533,NaN,신내6대주,202309,신내로19길 42,1,0.203699


In [83]:
subway_train.to_csv('../data/interim/subway_train.csv', index=False)
subway_test.to_csv('../data/interim/subway_test.csv', index=False)